In [14]:
from time import sleep 
from selenium import webdriver 
import pandas as pd
from bs4 import BeautifulSoup 
chrome_driver_path = r'C:\Users\ELITEBOOK 1030\Downloads\chromedriver-win64\chromedriver.exe'

#link query 
def get_url(Departing, Arrival, Departure_date):
    TEMPLATES = 'https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort={}&arrPort={}&departureDate={}%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en'
    url = TEMPLATES.format(Departing, Arrival, Departure_date)
    return url
#State Variables 
a = input('''Choose your departure location from the following options:
ABJ>Abidjan,
ABV>Abuja, 
ACC>Accra ,
AKR>Akure ,
ANA>Anambra, 
BJL>Banjul ,
BNI>Benincity ,
CBQ>Calabar ,
COO>Cotonou ,
DSS>Dakar Blaise Diagne Internatio, 
DLA>Douala 
ENU>Enugu ,
GMO>Gombe ,
ILR>Ilorin ,
JED>Jeddah ,
JNB>Johannesburg, 
KAD>Kaduna ,
KAN>Kano ,
LOS>Lagos,
LFW>Lome ,
LGW>London, 
MIU>Maiduguri, 
MDI>Makurdi ,
ROB>Monrovia ,
BOM>Mumbai ,
QOW>Owerri ,
PHC>Portharcourt,
QUO>Uyo ,
QRW>Warri,
YOL>Yola 

Enter your departure location (e.g., ABV for Abuja): ''')

b = input('''Choose your arrival location from the following options:
ABJ>Abidjan,
ABV>Abuja, 
ACC>Accra ,
AKR>Akure ,
ANA>Anambra, 
BJL>Banjul ,
BNI>Benincity ,
CBQ>Calabar ,
COO>Cotonou ,
DSS>Dakar Blaise Diagne Internatio, 
DLA>Douala 
ENU>Enugu ,
GMO>Gombe ,
ILR>Ilorin ,
JED>Jeddah ,
JNB>Johannesburg, 
KAD>Kaduna ,
KAN>Kano ,
LOS>Lagos,
LFW>Lome ,
LGW>London, 
MIU>Maiduguri, 
MDI>Makurdi ,
ROB>Monrovia ,
BOM>Mumbai ,
QOW>Owerri ,
PHC>Portharcourt,
QUO>Uyo ,
QRW>Warri,
YOL>Yola 

Enter your arrival location (e.g., LOS for Lagos): ''')

c = input('''DEPARTURE DATE
          
ENTER YOUR DEPARTURE DATE (e.g., dd.mm.yyyy): ''') # time format

link = get_url(a, b, c)
print(link)   # link display

# Initialize WebDriver
driver = webdriver.Chrome(chrome_driver_path)
driver.get(link)
sleep(10)

flight_rows = driver.find_elements_by_xpath(
    '//div[@class="row w-100 no-gutters"]')
print(flight_rows)

#CREATE EMPTY LISTS
depart_time_list = []
depart_loc_list = []
depart_date_list = []
#
flight_duration_list = []
total_stop_list = []
flight_no_list = []
#
arrival_time_list = [] 
arrival_loc_list = []
arrival_date_list = []  
#
economy_price_list=[]   
executive_economy_price_list=[] 
business_price_list = []

#PARSE THE LINK IN OTHER TO GET THE OUTERHTML
for WebElement in flight_rows:
    elementHTML = WebElement.get_attribute('outerHTML')
    elementsoup = BeautifulSoup(elementHTML, 'html.parser') 

    #FLIGHT INFO BOX 1
    flight_box1= elementsoup.find('div',{"class": "info-block"})
    depart_time = flight_box1.find("span",{"class": "time"})
    depart_loc = flight_box1.find("span",{"class": "port"})
    depart_date = flight_box1.find("span",{"class": "date"})
    depart_time_list.append(depart_time.text)
    depart_loc_list.append(depart_loc.text)
    depart_date_list.append(depart_date.text)

    
    #FLIGHT INFO BOX 2
    flight_box2= elementsoup.find('div',{"class": "info-row"})
    flight_duration = flight_box2.find("span",{"class": "flight-duration"})
    total_stop = flight_box2.find("span",{"class": "total-stop"})
    flight_no = flight_box2.find("span",{"class": "flight-no"})
    flight_duration_list.append(flight_duration.text)
    total_stop_list.append(total_stop.text)
    flight_no_list.append(flight_no.text)


    #FLIGHT INFO BOX 3
    flight_box3= elementsoup.find('div',{"class": "info-block text-right"})
    arrival_time =   flight_box3.find("span",{"class": "time"})
    arrival_loc =   flight_box3.find("span",{"class": "port"})
    arrival_date =   flight_box3.find("span",{"class": "date"})
    arrival_time_list.append(arrival_time.text)
    arrival_loc_list.append(arrival_loc.text)
    arrival_date_list.append(arrival_date.text)

    
    #FLIGHT INFO BOX 4 (MONEY SECTION)
    flight_box4 = elementsoup.find('div', {'class': "desktop-fare-block"})   # FIRST CONTAINER 

    # Attempt to find economy_price using class "currency-best-offer"
    economy_price = flight_box4.find("span", {"class": "currency-best-offer"})
    try:
        if economy_price is not None:
            economy_price_text = economy_price.text.strip()
            economy_price_list.append(economy_price_text)
        else:
        # If economy_price is None, try to find it using class "currency"
            economy_price = flight_box4.find("span", {"class": "currency"})
            if economy_price is not None:
                economy_price_text = economy_price.text.strip()
                economy_price_list.append(economy_price_text)
            else:
                economy_price_list.append("no seat")
    except Exception as e:
        print("An error occurred:", e)

     #OBTAINING EXECUTIVE PRICE AND PRINTING "NO SEAT" IS ITS EMPTY(NONE)
    executive_economy_price =   flight_box4.find("span",{"class": "no-seat-text"})
    try:
        if executive_economy_price is not None:
            executive_economy_price_text = executive_economy_price.text.strip()
            executive_economy_price_list.append(executive_economy_price_text)
        else:
            economy_price_list.append("no seat")
    except  Exception as ee:
        print("An error in executive_economy_price occured",ee)   
    #OBTAINING BUSINESS PRICE AND PRINTING "NO SEAT" IS ITS EMPTY(NONE) 
    business_price = flight_box4.find("span",{"class": "currency"})
    if business_price is not None:
        business_price_text = business_price.text.strip()
        business_price_list.append(business_price_text)
    else:
        business_price_list.append("no seat")
        
    
# print(depart_time_list )
# print(depart_loc_list )
# print(depart_date_list) 
# #
# sleep(5)

# print(flight_duration_list) 
# print(total_stop_list) 
# print(flight_no_list)  
# #
# sleep(5)

# print(arrival_time_list) 
# print(arrival_loc_list )
# print(arrival_date_list)  
# #
# sleep(5)

# print(economy_price_list)
# print(executive_economy_price_list)
# print(business_price_list)
        
# TABLE
Table ={'FLIGHT DEPART':depart_loc_list,'FLIGHT ARRIVAL':arrival_loc_list,'DEPART TIME':depart_time_list,'ARRIVAL TIME':arrival_time_list,'DEPART DATE':depart_date_list,'ARRIVAL DATE':arrival_date_list,'FLIGHT DURATION':flight_duration_list,'TOTAL STOP':total_stop_list,'FLIGHT NO':flight_no_list,'BUSINESS PRICE':business_price_list,'ECONOMY PRICE':economy_price_list,'EXECUTIVE ECONOMY PRICE':executive_economy_price_list}
df = pd.DataFrame(Table)
df



https://book-airpeace.crane.aero/ibe/availability?tripType=ONE_WAY&depPort=ABV&arrPort=LOS&departureDate=03.03.2024%20%20%20%20%20%20%20%20&adult=1&child=0&infant=0&lang=en
[<selenium.webdriver.remote.webelement.WebElement (session="77a9fdb22231d119c236256f627202ac", element="f.E81244E21372998F4A222393E4EBA400.d.6EB454ACF982C454054E024D4F828819.e.108")>, <selenium.webdriver.remote.webelement.WebElement (session="77a9fdb22231d119c236256f627202ac", element="f.E81244E21372998F4A222393E4EBA400.d.6EB454ACF982C454054E024D4F828819.e.109")>, <selenium.webdriver.remote.webelement.WebElement (session="77a9fdb22231d119c236256f627202ac", element="f.E81244E21372998F4A222393E4EBA400.d.6EB454ACF982C454054E024D4F828819.e.110")>, <selenium.webdriver.remote.webelement.WebElement (session="77a9fdb22231d119c236256f627202ac", element="f.E81244E21372998F4A222393E4EBA400.d.6EB454ACF982C454054E024D4F828819.e.111")>, <selenium.webdriver.remote.webelement.WebElement (session="77a9fdb22231d119c236256f627202ac", 

,FLIGHT DEPART,FLIGHT ARRIVAL,DEPART TIME,ARRIVAL TIME,DEPART DATE,ARRIVAL DATE,FLIGHT DURATION,TOTAL STOP,FLIGHT NO,BUSINESS PRICE,ECONOMY PRICE,EXECUTIVE ECONOMY PRICE
0,Abuja (ABV),Lagos (LOS),06:30,07:50,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7131,$ 109.00,$ 81.10,No Seat
1,Abuja (ABV),Lagos (LOS),08:30,09:50,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7121,$ 109.00,$ 81.10,No Seat
2,Abuja (ABV),Lagos (LOS),12:35,13:55,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7125,$ 109.00,$ 81.10,No Seat
3,Abuja (ABV),Lagos (LOS),14:00,15:20,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7445,$ 135.80,$ 81.10,No Seat
4,Abuja (ABV),Lagos (LOS),16:20,17:40,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7227,$ 109.00,$ 81.10,No Seat
5,Abuja (ABV),Lagos (LOS),19:25,20:45,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7429,no seat,$ 81.10,No Seat
6,Abuja (ABV),Lagos (LOS),20:00,21:20,03 Mar 2024,03 Mar 2024,1h 20m,Nonstop,P4-7139,$ 109.00,$ 81.10,No Seat
